In [2]:
import numpy as np
import pandas as pd

from plotnine import *
from sqlalchemy import create_engine

In [3]:
eng = create_engine('mysql://127.0.0.1/sms_fb?read_default_file=~/.my.cnf&charset=utf8mb4')

In [4]:
sql = """
SELECT activity, age, gender, android_check, da.fb_user_id, sms_user_id
FROM baseline_data_targ
JOIN data_activity da
ON activity = record_idn
WHERE da.fb_user_id in (SELECT DISTINCT fb_user_id FROM fb_posts)
OR sms_user_id in (SELECT DISTINCT userId FROM sms_data)
"""
baseline_raw = pd.read_sql(sql, eng)
baseline = baseline_raw.copy()
baseline.head()

,activity,age,gender,android_check,fb_user_id,sms_user_id
0,R_29gYxinH5R1iJpQ,30.0,2.0,2.0,10158757293721897,None
1,R_1Nsmw9Pf8xLdMAK,29.0,1.0,2.0,10219344023491608,None
2,R_2axIFcDreGO9A1E,29.0,1.0,2.0,10164291780836337,None
3,R_XH9VGGqTNJHzuSJ,53.0,1.0,2.0,3811016502253166,R_XH9VGGqTNJHzuSJ
4,R_1oupORlxMUhXkby,32.0,2.0,2.0,474411093965925,R_1oupORlxMUhXkby


In [5]:
baseline.groupby('android_check')['android_check'].count()

android_check
1.0    792
2.0    344
Name: android_check, dtype: int64

In [6]:
median_age = baseline.age.median()
median_age

36.0

In [7]:
baseline['is_young'] = np.where(baseline['age'] < median_age, 1, 0)
baseline['gender'] = np.where(baseline['gender'] == 1.0, 'M', 'F')
baseline['android_check'] = np.where(baseline['android_check'] == 1.0, 'iPhone', 'Android')
baseline.head()

,activity,age,gender,android_check,fb_user_id,sms_user_id,is_young
0,R_29gYxinH5R1iJpQ,30.0,F,Android,10158757293721897,None,1
1,R_1Nsmw9Pf8xLdMAK,29.0,M,Android,10219344023491608,None,1
2,R_2axIFcDreGO9A1E,29.0,M,Android,10164291780836337,None,1
3,R_XH9VGGqTNJHzuSJ,53.0,M,Android,3811016502253166,R_XH9VGGqTNJHzuSJ,0
4,R_1oupORlxMUhXkby,32.0,F,Android,474411093965925,R_1oupORlxMUhXkby,1


In [8]:
distribs = baseline.groupby(['android_check', 'gender', 'is_young'], as_index=False)['activity'].count()
distribs

,android_check,gender,is_young,activity
0,Android,F,0,157
1,Android,F,1,98
2,Android,M,0,52
3,Android,M,1,37
4,iPhone,F,0,270
5,iPhone,F,1,344
6,iPhone,M,0,94
7,iPhone,M,1,84


In [9]:
distribs.groupby('android_check')['activity'].sum()

android_check
Android    344
iPhone     792
Name: activity, dtype: int64

In [10]:
distribs['total'] = np.where(distribs['android_check'] == 'Android', 350, 800)
distribs['frac'] = distribs['activity'] / distribs['total']
distribs

,android_check,gender,is_young,activity,total,frac
0,Android,F,0,157,350,0.448571
1,Android,F,1,98,350,0.280000
2,Android,M,0,52,350,0.148571
3,Android,M,1,37,350,0.105714
4,iPhone,F,0,270,800,0.337500
5,iPhone,F,1,344,800,0.430000
6,iPhone,M,0,94,800,0.117500
7,iPhone,M,1,84,800,0.105000


In [11]:
def match_sample(baseline_iphone, baseline_android):
	for gender in ['F', 'M']:
		for is_young in [1, 0]:
			android_num = baseline_android.loc[(baseline_android['gender'] == gender) & (baseline_android['is_young'] == is_young), :].shape[0]
			iphone_samp = baseline_iphone.loc[(baseline_iphone['gender'] == gender) & (baseline_iphone['is_young'] == is_young), :].sample(android_num)
			yield iphone_samp

In [12]:
baseline.shape

(1136, 7)

In [13]:
baseline_android = baseline.loc[baseline['android_check'] == 'Android', :]

baseline_iphone = baseline.loc[baseline['android_check'] == 'iPhone', :]
baseline_iphone_downsamp = pd.concat([samp for samp in match_sample(baseline_iphone, baseline_android)])

baseline_matched = pd.concat([baseline_android, baseline_iphone_downsamp])
baseline_matched.groupby(['android_check', 'gender', 'is_young'], as_index=False)['activity'].count()

,android_check,gender,is_young,activity
0,Android,F,0,157
1,Android,F,1,98
2,Android,M,0,52
3,Android,M,1,37
4,iPhone,F,0,157
5,iPhone,F,1,98
6,iPhone,M,0,52
7,iPhone,M,1,37


In [14]:
activities = ['"{}"'.format(activity) for activity in baseline_matched.activity.to_numpy()]
df = pd.read_sql('SELECT * FROM baseline_data_targ WHERE activity IN ({})'.format(', '.join(activities)), eng)
df.shape

(688, 209)

In [15]:
df.groupby('android_check')['android_check'].count()

android_check
1.0    344
2.0    344
Name: android_check, dtype: int64

In [17]:
df['has_fb'] = np.where(pd.notnull(df['fb_user_id']), 1, 0)
df.loc[df.android_check == 2.0, :].groupby('has_fb')['has_fb'].count()

has_fb
0    155
1    189
Name: has_fb, dtype: int64

In [18]:
activities_android = ['"{}"'.format(activity) for activity in baseline_android.activity.to_numpy()]
activities_iphone = ['"{}"'.format(activity) for activity in baseline_iphone_downsamp.activity.to_numpy()]
with eng.connect() as con:
	con.execute('CREATE TABLE baseline_android AS SELECT * FROM baseline_data_targ WHERE activity IN ({})'.format(', '.join(activities_android)))
	con.execute('CREATE TABLE baseline_iphone AS SELECT * FROM baseline_data_targ WHERE activity IN ({})'.format(', '.join(activities_iphone)))